In [ ]:
# Install packages
!pip install nltk
!pip install transformers
!pip install torch torchvision torchaudio

Possbilità anche di utilizzare un tokenizzatore grazie a spacy o NLTK

In [ ]:
import nltk
import re
import numpy as np
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
#model = AutoModel.from_pretrained("medicalai/ClinicalBERT")

nltk.download('stopwords')  # Download nltk stopwords
#nltk.download('punkt')      # Download nltk punkt tokenizer
#nltk.download('wordnet')

text = []
stopwords_list = set(stopwords.words("english"))

file_path = r'C:\Users\Utente\Desktop\Semantic Textual Similarity\Clinical Semantic Textual Similarity\clinicalSTS2019\train\clinicalSTS2019.train.txt'


with open(file_path, 'r') as f:
    for line in f.readlines():

        # Splitting by tab
        line_parts = re.split('\t', line)
        
        filtered_sentences = []
        for sentence in line_parts:
            # Lowercasing
            lower_sentence = sentence.lower()

            remove_apex = lower_sentence.replace ('"', ' ')
            
            remove_whitespace = remove_apex.strip()
            
            tokenized_sentence = tokenizer.tokenize(remove_whitespace)
            
            # Remove stopwords
            filtered_tokens = [token for token in tokenized_sentence if token.lower() not in stopwords_list]

            filtered_sentences.append(filtered_tokens)

        text.append(filtered_sentences)

Viene inserito la tokenizzazione utilizzando un modello preaddestrato. Non restituisce risultati sperati. Non viene eseguita alcuna tecnica di normalizzazione (lemmatizzazione o stemming)

In [ ]:
def make_split(embedded_representation, labels):
    X_train, X_test, y_train, y_test = train_test_split(embedded_representation, labels, random_state=104, test_size=0.25, shuffle=True)

    y_train = np.array(y_train, dtype=object)
    y_test = np.array(y_test, dtype=object)

    X_train = np.array(X_train, dtype=object)
    X_test = np.array(X_test, dtype=object)

    return X_train, X_test, y_train, y_test

In [ ]:
def cosine_similarity_unidimensional(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    similarity = dot_product / (norm_v1 * norm_v2)
    return similarity

In [ ]:
def normalize_train(y):
    y = [float(val) for val in y]  # Converti i valori in numeri decimali
    min_val = np.min(y)
    max_val = np.max(y)
    normalized_labels_train = (y - min_val) / (max_val - min_val) * 2 - 1

    return normalized_labels_train